In [1]:
# all things geodesic including transformations between standard 
# geospatial coordinate systems
using Geodesy

# for direct manipulations of Cartesian and Longitude-Latitude coordinates
using CoordinateTransformations, Rotations, StaticArrays


include("mathies.jl")




0.3141592653589793

In [2]:
pio10

0.3141592653589793

In [4]:
# generate the coordinates for global positioning centerpoints (beads)
#   icosohedral vertices are centerpoints of the 12 dodecahedral regions 


# icoso_verts points have an edge not a vertex at top and bottom;
# rotate -31.71 degrees ( -( pi/2 - atan(Φ) ) ) on the X axis
rotateX  = LinearMap( RotX( -( pi/2 - atan(Φ)) ) )
# that results in untidy longitudes; spin eastward 18 degrees
rotateZ  = LinearMap( RotZ(pi/10) )
# create a composite rotation
rotate = rotateZ ∘ rotateX 
points = SVector.(rotate.(GEO_ICOSO_VERTS))



# generate LLA's for each point


# radius here is the distance of the vertex from the icosohedron center
icoso_circumradius = sqrt(Φ^2 + 1)

# latitude is asin(z / R)
#   with rounding to prevent error in asin when |z/r| > 1
bead_lats = asin.( round(point[3]/icoso_circumradius, sigdigits=9) for point in points)

# longitude is atan2(y, x)
bead_lons = [atan(point[2],point[1]) for point in points]

degrees_lat = round.(rad2deg.(bead_lats), sigdigits=7)
degrees_lon = round.(rad2deg.(bead_lons))

earth12_labels = ["MA","BA","DA","LA","KA","SA","ME","BE","DE","LE","KE","SE"]

struct Bead
    location::LLA
    radius::Float64
    label::String
    description::String
end

expansion = 1 + 2π/5
contraction = 1 / (1 + 2π/5)

earth12 = []

for i in 1:12    
    push!(earth12, Bead(LLA(degrees_lat[i],  degrees_lon[i]),
                        contraction * earth_radius,
                        earth12_labels[i],
                        "point " * string(i) * " of 12 equidistant points"))
end
earth12


12-element Array{Any,1}:
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MA", "point 1 of 12 equidistant points")      
 Bead(LLA(lat=-26.56505°, lon=-72.0°, alt=0.0), 2.826390255215226e6, "BA", "point 2 of 12 equidistant points") 
 Bead(LLA(lat=-26.56505°, lon=144.0°, alt=0.0), 2.826390255215226e6, "DA", "point 3 of 12 equidistant points") 
 Bead(LLA(lat=-26.56505°, lon=72.0°, alt=0.0), 2.826390255215226e6, "LA", "point 4 of 12 equidistant points")  
 Bead(LLA(lat=-26.56505°, lon=-144.0°, alt=0.0), 2.826390255215226e6, "KA", "point 5 of 12 equidistant points")
 Bead(LLA(lat=-26.56505°, lon=0.0°, alt=0.0), 2.826390255215226e6, "SA", "point 6 of 12 equidistant points")   
 Bead(LLA(lat=90.0°, lon=-90.0°, alt=0.0), 2.826390255215226e6, "ME", "point 7 of 12 equidistant points")      
 Bead(LLA(lat=26.56505°, lon=108.0°, alt=0.0), 2.826390255215226e6, "BE", "point 8 of 12 equidistant points")  
 Bead(LLA(lat=26.56505°, lon=-36.0°, alt=0.0), 2.826390255215226e6, "DE", "poin

In [20]:
# pi/10 is 18 degrees, 3 * pi/10 is 54
# in geographic terms, the first element of each pair is North (Z+), the second is East (Y+)
directions = SVector.([ ( 0           ,  0            ), 
                        ( sin(pi/10)  , -cos(pi/10)   ), 
                        (-sin(3*pi/10), -cos(3*pi/10) ), 
                        (-sin(3*pi/10),  cos(3*pi/10) ),
                        ( sin(pi/10)  ,  cos(pi/10)   ), 
                        ( Φ           ,  0            )])

# these directions help form transformations for expanding and and contracting an area;
# nothing gets changed on X; Y is the East/second element; Z is the North/first element
transformations = [] 
for direction in directions
    push!(transformations, 
        LinearMap(RotZ(direction[1])) ∘ LinearMap(RotY(direction[2])))
end
# each point has its latitude and longitude calculated relative to the original point and the new radius 

labels = ["M","B","D","L", "K", "S"]
function contract(bead::Bead)
    beads = []
    #radius = bead.radius * contraction
    radius = bead.radius # * contraction

    for i in 1:6         
        push!(beads, Bead(bead.location, radius, bead.label * labels[i], "" ))
    end
    return beads 
end

contract(earth12[2])
transformations
#directions
bead=earth12[2]
"""
for transform in transformations
    println(bead.location ) #* transform)
end
    
"""
earth12


earth12

In [11]:
contract(earth12[1])

6-element Array{Any,1}:
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAM", "")
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAB", "")
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAD", "")
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAL", "")
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAK", "")
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MAS", "")

In [93]:
myspot = LLA( 30.228711, 92.027100 )
ced404 = LLA( 30.228678, 92.027272 )
ber266 = LLA( 30.204461, 92.031397 )


using Printf
@printf "%.4f" distance(ced404, ber266)/1000

# for 3-D transformation; put into cartesian coordinates 
myspotECEF = ECEF( myspot, wgs84 )



2.7138

ECEF(-195096.58265667254, 5.512084465685038e6, 3.1923052497591996e6)